In [5]:
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# import torch

# # Load pretrained model and tokenizer
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model.eval()  # Set the model to evaluation mode

# # Example text
# text = "Hello, my dog is cute"

# # Tokenize text
# inputs = tokenizer(text, return_tensors="pt")

# # Perform inference
# with torch.no_grad():
#     logits = model(**inputs).logits

# # Decode the prediction
# predicted_class_id = logits.argmax().item()
# prediction = model.config.id2label[predicted_class_id]

# print(f"Predicted class id: {predicted_class_id}, Prediction: {prediction}")


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# import torch

# # Load the tokenizer and the model
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# model.eval()  # Set the model to evaluation mode

# # Prepare a sample input for the export
# sample_text = "Hello, my dog is cute"
# inputs = tokenizer(sample_text, return_tensors="pt")

# # Export the model
# torch.onnx.export(model,               # model being run
#                   (inputs['input_ids'],),  # model input (or a tuple for multiple inputs)
#                   "distilbert.onnx",   # where to save the model (can be a file or file-like object)
#                   export_params=True,  # store the trained parameter weights inside the model file
#                   opset_version=11,    # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names=['input_ids'],   # the model's input names
#                   output_names=['output'],  # the model's output names
#                   dynamic_axes={'input_ids': {0: 'batch_size'},  # variable length axes
#                                 'output': {0: 'batch_size'}})
